In [3]:
from transformers import (
    BertModel, 
    AutoConfig, 
    AutoTokenizer, 
    Trainer,
    TrainingArguments
    )
import torch.nn as nn
import datasets
import csv


In [33]:
import torch
import numpy as np

torch.cuda.is_available()

True

In [5]:

model_name = "bert-base-uncased"
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)

config.problem_type = 'regression'

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [123]:

# Add linear layer
output_size = 1  


# Combine BERT and the linear layer
class BertWithLinear(nn.Module):
    def __init__(self):
        super(BertWithLinear, self).__init__()
        self.bert = bert_model.to('cuda')
        self.ft = nn.Sequential(
            nn.Linear(config.hidden_size, 512),
            nn.Dropout(0.4),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.Dropout(0.4),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.Dropout(0.4),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(16, output_size)


        ).to('cuda')
        

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None, labels=None):
        output = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, position_ids=position_ids, head_mask=head_mask, inputs_embeds=inputs_embeds)
        # Use pooled output for classification/regression
        pooled_output = output.pooler_output
        return self.ft(pooled_output)

model = BertWithLinear().to('cuda')

In [124]:
# freeze BERT pretrained weights
# for param in model.bert.embeddings.parameters():
#     param.requires_grad = False
model = nn.DataParallel(model)

In [84]:
# Load dataset
feats_fp = open("BERT_X.csv", "r")
labels_fp = open("BERT_y.csv", "r")
feats = csv.reader(feats_fp)
labels = csv.reader(labels_fp)

# skip header
next(feats)
next(labels)

unscaled_data = {'text': [], 'label': []}
for row in feats:
    unscaled_data['text'].append(row[0].strip().replace("\n", " "))
for row in labels:
    unscaled_data['label'].append(float(row[0].strip().replace("\n", "")))
    

In [96]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_labels = scaler.fit_transform(np.array(data['label']).reshape(-1,1)).flatten()
scaled_data = unscaled_data
scaled_data['label'] = scaled_labels
scaled_labels

array([-0.58786439, -0.94649659,  2.86819603, ..., -1.04441134,
       -0.86218111, -0.69471581])

In [97]:
print(len(scaled_data['text']), len(scaled_data['label']))
assert len(scaled_data['text']) == len(scaled_data['label'])
dataset = datasets.Dataset.from_dict(scaled_data)

26990 26990


In [100]:
print(dataset[0]['text'])
scaler.inverse_transform(np.array([dataset[0]['label']]).reshape(-1,1))

Overview  HearingLife is a national hearing care company and part of the Demant Group, a global leader in hearing healthcare built on a heritage of care, health, and innovation since 1904. HearingLife operates more than 600 hearing care centers across 42 states. We follow a scientific, results-oriented approach to hearing healthcare that is provided by highly skilled and caring professionals. Our vision is to help more people hear better through life-changing hearing health delivered by the best personalized care. This Team Member must uphold the HearingLife Core Values:   We create trust  We are team players  We apply a can-do attitude  We create innovative solutions   Responsibilities  You will help more people hear better by providing clinical expertise to diagnose and treat hearing loss while ensuring a positive patient experience. The Hearing Care Provider acts in accordance with required industry and state professional licensing standards and local practice scope and is responsib

array([[63000.]])

In [101]:
# max salary

max_salary = max(data['label'])
min_salary = min(data['label'])
print(max_salary, min_salary)


285000.0 22000.0


In [102]:
# Tokenize the dataset
tokenizer = AutoTokenizer.from_pretrained(model_name)
def preprocess(examples):
    return tokenizer(examples['text'], truncation=True, padding=True)

tokenized_dataset = dataset.map(preprocess, batched=True)

  0%|          | 0/27 [00:00<?, ?ba/s]

In [13]:
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=16,  
    learning_rate=5e-5,               
    warmup_steps=500,                
    weight_decay=0.01,              
    logging_dir='./logs',
)


In [31]:

trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=tokenized_dataset,    
    eval_dataset=tokenized_dataset,
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertWithLinear.forward` and have been ignored: text. If text are not expected by `BertWithLinear.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 26990
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 633


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/mnt/home/bhatta70/.local/lib/python3.6/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/mnt/home/bhatta70/.local/lib/python3.6/site-packages/torch/nn/modules/module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "<ipython-input-26-96aa96028594>", line 27, in forward
    output = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, position_ids=position_ids, head_mask=head_mask, inputs_embeds=inputs_embeds)
  File "/mnt/home/bhatta70/.local/lib/python3.6/site-packages/torch/nn/modules/module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/opt/software/Python/3.6.4-foss-2018a/lib/python3.6/site-packages/transformers/models/bert/modeling_bert.py", line 1006, in forward
    return_dict=return_dict,
  File "/mnt/hom

In [14]:
tokenized_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 26990
})

In [104]:
from torch.utils.data import Dataset

class RegressionDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __getitem__(self, idx):
        item = {
            'input_ids': torch.tensor(self.input_ids[idx]),
            'attention_mask': torch.tensor(self.attention_mask[idx]),
            'labels': torch.tensor(self.labels[idx])
        }
        return item

    def __len__(self):
        return len(self.labels)  # Assuming all data entries have labels

    
    

In [131]:
input_ids = tokenized_dataset['input_ids']
attention_mask = tokenized_dataset['attention_mask']
labels = dataset['label']  # Assuming your labels are in the original dataset

reg_dataset = RegressionDataset(input_ids, attention_mask, labels)
dataloader = torch.utils.data.DataLoader(reg_dataset, batch_size=32, shuffle=True)



In [132]:
model.parameters()

<generator object Module.parameters at 0x2af9cff8f5c8>

In [133]:
loss_fn = nn.MSELoss()  # Mean Squared Error is common for regression
optimizer = torch.optim.Adam([
    {'params': model.module.bert.parameters(), 'lr': 1e-8}, 
    {'params': model.module.ft.parameters() , 'lr' : 1e-8}   # standard lr for our NN
])
num_epochs = 3


# for epoch in range(num_epochs):
#     print(epoch,"/",num_epochs)
#     b=0
#     for batch in dataloader:

    
    
from torch.utils.data import random_split 

train_size = int(0.8 * len(reg_dataset))  # 80% of the dataset for training
val_size = len(reg_dataset) - train_size 

# Create the train and validation datasets
train_dataset, val_dataset = random_split(reg_dataset, [train_size, val_size])

# Create DataLoaders
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)  # No need to shuffle validation

# Modify your training loop (add validation)
for epoch in range(num_epochs):
    print(epoch,"/",num_epochs)

    ## Training Phase
    model.train()  # Set model to training mode
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to('cuda') 
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda')

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.squeeze(1), labels) # Ensure outputs are single-dimensional
        loss.backward()
        optimizer.step() 
        print(f"batch {b} complete. Loss: ",loss.item())
        b+=1
    ## Validation Phase
    model.eval()   # Set model to evaluation mode
    val_loss = 0
    with torch.no_grad():  # Disable gradient calculation for validation
        for batch in val_dataloader:
            # ... (Similar to training, but no optimizer updates)
            outputs = model(input_ids, attention_mask=attention_mask)
            val_loss += loss_fn(outputs.squeeze(1), labels).item()

    val_loss /= len(val_dataloader)
    print(f"Epoch {epoch} Validation Loss: {val_loss}")


0 / 3
batch 82 complete. Loss:  1.1621150970458984
batch 83 complete. Loss:  0.8956880569458008
batch 84 complete. Loss:  0.8037869930267334
batch 85 complete. Loss:  0.9169962406158447
batch 86 complete. Loss:  0.8884814977645874
batch 87 complete. Loss:  0.9881263971328735
batch 88 complete. Loss:  0.9900961518287659
batch 89 complete. Loss:  0.7278387546539307
batch 90 complete. Loss:  0.8394187688827515
batch 91 complete. Loss:  1.8925304412841797
batch 92 complete. Loss:  1.0616230964660645
batch 93 complete. Loss:  0.6486200094223022
batch 94 complete. Loss:  0.6075191497802734
batch 95 complete. Loss:  0.7383300065994263
batch 96 complete. Loss:  1.0172334909439087
batch 97 complete. Loss:  1.1043998003005981
batch 98 complete. Loss:  1.0885484218597412
batch 99 complete. Loss:  0.528542160987854
batch 100 complete. Loss:  0.4628363847732544
batch 101 complete. Loss:  0.7179670333862305
batch 102 complete. Loss:  0.9117007255554199
batch 103 complete. Loss:  0.5826869010925293
b

KeyboardInterrupt: 

In [146]:
# inference
text = "The responsibilities of this position include collaboration with business stakeholders to identify business opportunities and requirements to develop associated AI/ML models that deliver useful, actionable recommendations, or which augment human capabilities in various work settings. The candidate is responsible for modeling complex business and engineering problems, discovering insights and deploying AI capabilities using machine learning and associated feature engineering techniques. Responsibilities also include tracking projects and driving work on them through to completion, as well as implementing technical best practices from the fields of machine learning and software engineering."
inputs = tokenizer(text, return_tensors='pt').to('cuda')
outputs = model(**inputs)
scaler.inverse_transform(np.array([outputs.item()]))

array([93420.34047078])

In [114]:
del model
torch.cuda.empty_cache()